In [1]:
import pandas as pd
import sqlite3

In [3]:
# Step 1: Load Excel Files

coverage = pd.read_excel("coverage-data.xlsx")
incidence = pd.read_excel("incidence-rate-data.xlsx")
reported = pd.read_excel("reported-cases-data.xlsx")
vaccine_intro = pd.read_excel("vaccine-introduction-data.xlsx")
vaccine_schedule = pd.read_excel("vaccine-schedule-data.xlsx")

In [5]:
#Step 2: Data Cleaning & Normalization

def clean_columns(df):
    """Standardize column names to lowercase with underscores"""
    df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
    return df

In [7]:
# Apply cleaning
coverage = clean_columns(coverage)
incidence = clean_columns(incidence)
reported = clean_columns(reported)
vaccine_intro = clean_columns(vaccine_intro)
vaccine_schedule = clean_columns(vaccine_schedule)

In [9]:
# Handle missing values (basic strategy: drop rows with all NaN, fill others)
coverage = coverage.dropna(how="all").fillna(0)
incidence = incidence.dropna(how="all").fillna(0)
reported = reported.dropna(how="all").fillna(0)
vaccine_intro = vaccine_intro.dropna(how="all").fillna("Unknown")
vaccine_schedule = vaccine_schedule.dropna(how="all").fillna("Unknown")

In [11]:
# Ensure correct dtypes
for df in [coverage, incidence, reported, vaccine_intro, vaccine_schedule]:
    if "year" in df.columns:
        df["year"] = df["year"].astype(int, errors="ignore")

In [13]:
# Step 3: SQL Database Setup

In [17]:
# Create SQLite DB
conn = sqlite3.connect("vaccination_project.db")
cursor = conn.cursor()

In [19]:
# Drop tables if exist (for reruns)
tables = ["coverage", "incidence", "reported_cases", "vaccine_introduction", "vaccine_schedule"]
for t in tables:
    cursor.execute(f"DROP TABLE IF EXISTS {t}")

In [21]:
# Create tables
cursor.execute("""
CREATE TABLE coverage (
    group_name TEXT,
    code TEXT,
    name TEXT,
    year INTEGER,
    antigen TEXT,
    antigen_description TEXT,
    coverage_category TEXT,
    coverage_category_description TEXT,
    target_number INTEGER,
    dodge INTEGER,
    coverage REAL
)
""")

In [23]:
cursor.execute("""
CREATE TABLE incidence (
    group_name TEXT,
    code TEXT,
    name TEXT,
    year INTEGER,
    disease TEXT,
    disease_description TEXT,
    denominator INTEGER,
    incidence_rate REAL
)
""")

In [25]:
cursor.execute("""
CREATE TABLE reported_cases (
    group_name TEXT,
    code TEXT,
    name TEXT,
    year INTEGER,
    disease TEXT,
    disease_description TEXT,
    cases INTEGER
)
""")

In [27]:
cursor.execute("""
CREATE TABLE vaccine_introduction (
    iso_3_code TEXT,
    country_name TEXT,
    who_region TEXT,
    year INTEGER,
    description TEXT,
    intro TEXT
)
""")

In [29]:
cursor.execute("""
CREATE TABLE vaccine_schedule (
    iso_3_code TEXT,
    country_name TEXT,
    who_region TEXT,
    year INTEGER,
    vaccine_code TEXT,
    vaccine_description TEXT,
    schedule_rounds TEXT,
    target_pop TEXT,
    target_pop_description TEXT,
    geoarea TEXT,
    age_administered TEXT,
    source_comment TEXT
)
""")

In [35]:
# Rename 'group' column to 'group_name' (to avoid conflict with SQL keyword)
if "group" in coverage.columns:
    coverage.rename(columns={"group": "group_name"}, inplace=True)
if "group" in incidence.columns:
    incidence.rename(columns={"group": "group_name"}, inplace=True)
if "group" in reported.columns:
    reported.rename(columns={"group": "group_name"}, inplace=True)

In [39]:
# Step 2.5: Rename reserved columns

if "group" in coverage.columns:
    coverage.rename(columns={"group": "group_name"}, inplace=True)
if "group" in incidence.columns:
    incidence.rename(columns={"group": "group_name"}, inplace=True)
if "group" in reported.columns:
    reported.rename(columns={"group": "group_name"}, inplace=True)


In [45]:
print(vaccine_intro.columns.tolist())

['iso_3_code', 'countryname', 'who_region', 'year', 'description', 'intro']


In [49]:
print("✅ Data cleaned and loaded into vaccination_project.db successfully!")

✅ Data cleaned and loaded into vaccination_project.db successfully!


In [63]:
import sqlite3

conn = sqlite3.connect("vaccination_project.db")
cursor = conn.cursor()

# Get all tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables_in_db = cursor.fetchall()

print("✅ Tables in database:", tables_in_db)

conn.close()

✅ Tables in database: [('coverage',), ('incidence',), ('reported_cases',), ('vaccine_introduction',), ('vaccine_schedule',), ('country',)]


In [65]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("vaccination_project.db")

# Use only tables that actually exist
tables = ["coverage", "incidence", "reported_cases"]  # update after Step 1 output

for table in tables:
    df = pd.read_sql_query(f"SELECT * FROM {table}", conn)
    df.to_csv(f"{table}.csv", index=False)
    print(f"✅ Exported {table}.csv")

conn.close()

✅ Exported coverage.csv
✅ Exported incidence.csv
✅ Exported reported_cases.csv


In [67]:
# Export all cleaned DataFrames to CSV for Power BI
coverage.to_csv("coverage.csv", index=False)
incidence.to_csv("incidence.csv", index=False)
reported.to_csv("reported_cases.csv", index=False)
vaccine_intro.to_csv("vaccine_intro.csv", index=False)
vaccine_schedule.to_csv("vaccine_schedule.csv", index=False)

print("✅ Export complete: CSVs saved in your working directory.")

✅ Export complete: CSVs saved in your working directory.


In [69]:
import pandas as pd

files = [
    "coverage-data.xlsx",
    "incidence-rate-data.xlsx",
    "reported-cases-data.xlsx",
    "vaccine-introduction-data.xlsx",
    "vaccine-schedule-data.xlsx"
]

for f in files:
    xls = pd.ExcelFile(f)
    print(f"\n📂 {f} contains sheets: {xls.sheet_names}")
    # Show first 5 rows of the first sheet
    df = pd.read_excel(f, sheet_name=0)
    print(df.head())


📂 coverage-data.xlsx contains sheets: ['Data', 'Reference', 'Reference (ADMIN)', 'Reference (OFFICIAL)', 'Reference (WUENIC)', 'Reference (HPV)', 'Reference (PAB)']
       GROUP CODE   NAME    YEAR  ANTIGEN  \
0  COUNTRIES  ABW  Aruba  2023.0      BCG   
1  COUNTRIES  ABW  Aruba  2023.0      BCG   
2  COUNTRIES  ABW  Aruba  2023.0  DIPHCV4   
3  COUNTRIES  ABW  Aruba  2023.0  DIPHCV4   
4  COUNTRIES  ABW  Aruba  2023.0  DIPHCV5   

                                 ANTIGEN_DESCRIPTION COVERAGE_CATEGORY  \
0                                                BCG             ADMIN   
1                                                BCG          OFFICIAL   
2  Diphtheria-containing vaccine, 4th dose (1st b...             ADMIN   
3  Diphtheria-containing vaccine, 4th dose (1st b...          OFFICIAL   
4  Diphtheria-containing vaccine, 5th dose (2nd b...             ADMIN   

  COVERAGE_CATEGORY_DESCRIPTION  TARGET_NUMBER   DOSES  COVERAGE  
0       Administrative coverage            NaN     

In [71]:
import pandas as pd

# Load Excel files (always use 'Data' sheet)
coverage = pd.read_excel("coverage-data.xlsx", sheet_name="Data")
incidence = pd.read_excel("incidence-rate-data.xlsx", sheet_name="Data")
reported = pd.read_excel("reported-cases-data.xlsx", sheet_name="Data")
introduction = pd.read_excel("vaccine-introduction-data.xlsx", sheet_name="Data")
schedule = pd.read_excel("vaccine-schedule-data.xlsx", sheet_name="Data")

# Standardize column names (lowercase, underscores)
def clean_columns(df):
    df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
    return df

coverage = clean_columns(coverage)
incidence = clean_columns(incidence)
reported = clean_columns(reported)
introduction = clean_columns(introduction)
schedule = clean_columns(schedule)

# Quick check
print("Coverage:", coverage.head(2), "\n")
print("Incidence:", incidence.head(2), "\n")
print("Reported:", reported.head(2), "\n")
print("Introduction:", introduction.head(2), "\n")
print("Schedule:", schedule.head(2), "\n")

Coverage:        group code   name    year antigen antigen_description  \
0  COUNTRIES  ABW  Aruba  2023.0     BCG                 BCG   
1  COUNTRIES  ABW  Aruba  2023.0     BCG                 BCG   

  coverage_category coverage_category_description  target_number  doses  \
0             ADMIN       Administrative coverage            NaN    NaN   
1          OFFICIAL             Official coverage            NaN    NaN   

   coverage  
0       NaN  
1       NaN   

Incidence:        group code   name    year     disease          disease_description  \
0  COUNTRIES  ABW  Aruba  2023.0         CRS  Congenital rubella syndrome   
1  COUNTRIES  ABW  Aruba  2023.0  DIPHTHERIA                   Diphtheria   

                      denominator  incidence_rate  
0          per 10,000 live births             0.0  
1  per 1,000,000 total population             0.0   

Reported:        group code   name    year     disease          disease_description  \
0  COUNTRIES  ABW  Aruba  2023.0       

In [73]:
import sqlite3

# Create a new SQLite database (or connect if exists)
conn = sqlite3.connect("vaccination_project.db")

# Save each dataframe as a table
coverage.to_sql("coverage", conn, if_exists="replace", index=False)
incidence.to_sql("incidence", conn, if_exists="replace", index=False)
reported.to_sql("reported", conn, if_exists="replace", index=False)
introduction.to_sql("introduction", conn, if_exists="replace", index=False)
schedule.to_sql("schedule", conn, if_exists="replace", index=False)

conn.close()

print("Data saved to vaccination_project.db")

Data saved to vaccination_project.db


In [75]:
# Save all datasets to CSV for Power BI
coverage.to_csv("coverage.csv", index=False)
incidence.to_csv("incidence.csv", index=False)
reported.to_csv("reported.csv", index=False)
introduction.to_csv("introduction.csv", index=False)
schedule.to_csv("schedule.csv", index=False)

print(" All CSV files exported successfully! You can now import them into Power BI.")

 All CSV files exported successfully! You can now import them into Power BI.


In [77]:
for table in tables:
    df = pd.read_sql_query(f"SELECT * FROM {table}", conn)
    df.to_csv(f"{table}.csv", index=False)
    print(f" Exported {table}.csv")

ProgrammingError: Cannot operate on a closed database.